In [1]:
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer

model_ckpt = "bert-base-uncased"
#使用AutoTokenizer.from_pretrained("bert-base-uncased")初始化了一个与“bert-base-uncased”模型相对应的分词器，用于对文本进行预处理。
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

text = "time flies like an arrow"
#文本“time flies like an arrow”被分词成模型词汇表中的各个标记。
# return_tensors="pt"确保输出为PyTorch张量格式，而add_special_tokens=False表示不添加默认的特殊标记，如[CLS]和[SEP]。
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
print("输入文本对应的令牌ID序列。inputs",inputs)
print(inputs.input_ids)
#通过AutoConfig.from_pretrained("bert-base-uncased")加载了BERT模型的配置信息，包含词汇表大小、隐藏层维度等设置
config = AutoConfig.from_pretrained(model_ckpt)
#利用从配置文件中加载的信息，创建了一个嵌入层(nn.Embedding)。这个层的作用是将每个令牌ID映射到一个固定维度的向量（即嵌入向量），其维度等于config.hidden_size
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
print("嵌入层的配置信息token_emb:",token_emb)

inputs_embeds = token_emb(inputs.input_ids)
print("inputs_embeds:",inputs_embeds)
print(inputs_embeds.size())

输入文本对应的令牌ID序列。inputs {'input_ids': tensor([[ 2051, 10029,  2066,  2019,  8612]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
tensor([[ 2051, 10029,  2066,  2019,  8612]])
嵌入层的配置信息token_emb: Embedding(30522, 768)
inputs_embeds: tensor([[[ 1.5333,  1.4574,  0.8291,  ..., -0.6084, -0.9543, -0.7479],
         [-1.0927, -0.4465,  0.5279,  ..., -1.4830,  0.8137, -0.4334],
         [-0.3130,  0.1337,  0.1402,  ..., -0.1457,  0.4639,  0.2971],
         [ 1.6886, -0.6601,  0.7638,  ..., -0.9121,  0.2720,  1.0560],
         [-0.1156, -0.9799, -0.1914,  ...,  0.3400, -1.1827, -0.4565]]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([1, 5, 768])


In [4]:
import torch
from math import sqrt

Q = K = V = inputs_embeds
dim_k = K.size(-1)
print('dim_k:', dim_k)
# 使用torch.bmm函数执行批量矩阵乘法，
# 即Q与K的转置（交换最后一维和倒数第二维）相乘。这一步是计算自注意力中的对齐得分，衡量每个查询（Q中的一个位置）与所有键（K中的所有位置）之间的相关性。
scores = torch.bmm(Q, K.transpose(1,2)) / sqrt(dim_k)
print(scores)
print(scores.size())

dim_k: 768
tensor([[[28.5910,  0.6450, -1.3360, -0.0984,  0.9283],
         [ 0.6450, 27.0317,  0.2024, -0.6470,  0.8473],
         [-1.3360,  0.2024, 28.7352, -2.5307, -1.2083],
         [-0.0984, -0.6470, -2.5307, 28.5995,  0.6047],
         [ 0.9283,  0.8473, -1.2083,  0.6047, 27.1710]]],
       grad_fn=<DivBackward0>)
torch.Size([1, 5, 5])


In [6]:
import torch.nn.functional as F
# F.softmax()函数在这里被用来对scores张量进行softmax变换。
# Softmax函数是一个常用的激活函数，
# 它能将任意实数向量转换为概率分布，即所有元素的值域在0到1之间，并且所有元素之和为1。在这个上下文中，
# dim=-1指定了进行softmax操作的维度是scores张量的最后一个维度。这对于注意力机制而言至关重要，
# 因为它确保了每个查询位置对所有键位置的注意力权重总和为1，形成了一个有效的概率分布。
weights = F.softmax(scores, dim=-1)
print(weights)
print(weights.sum(dim=-1))

tensor([[[1.0000e+00, 7.2978e-13, 1.0066e-13, 3.4700e-13, 9.6877e-13],
         [3.4707e-12, 1.0000e+00, 2.2294e-12, 9.5350e-13, 4.2488e-12],
         [8.7141e-14, 4.0584e-13, 1.0000e+00, 2.6387e-14, 9.9016e-14],
         [3.4406e-13, 1.9880e-13, 3.0221e-14, 1.0000e+00, 6.9502e-13],
         [4.0080e-12, 3.6962e-12, 4.7318e-13, 2.8999e-12, 1.0000e+00]]],
       grad_fn=<SoftmaxBackward0>)
tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)


In [7]:
import torch
import torch.nn.functional as F
from math import sqrt

def scaled_dot_product_attention(query, key, value, query_mask=None, key_mask=None, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    if query_mask is not None and key_mask is not None:
        mask = torch.bmm(query_mask.unsqueeze(-1), key_mask.unsqueeze(1))
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -float("inf"))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

In [8]:
from torch import nn

class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, query, key, value, query_mask=None, key_mask=None, mask=None):
        attn_outputs = scaled_dot_product_attention(
            self.q(query), self.k(key), self.v(value), query_mask, key_mask, mask)
        return attn_outputs